In [ ]:
import random

import polars as pl
import pyarrow.parquet as pq

import src.pytorch.data.kaggle
import src.pytorch.models
import src.pytorch.models.utils
import src.visualization.performance

from src.pytorch.data.parquet import Dataset
from src.schemas.climsim import INPUT_COLUMNS, OUTPUT_COLUMNS

In [ ]:
# TRAINSET_DATA_PATH = "/home/data/subset_train.arrow"
TRAINSET_DATA_PATH = "/home/data/train.parquet"

# Common parameters
BATCH_SIZE = 3072
N_EPOCHS = 10
# Given TRAINING_SAMPLE_FRAC=0.7, 70% of the samples will be used for training
TRAINING_SAMPLE_FRAC = 0.7

# Training parameters
TRAINING_BUFFER_SIZE = 480
TRAINING_N_GROUP_PER_SAMPLING = 3
TRAINING_N_BATCH_PER_SAMPLING = 120

# Validation parameters
VALIDATION_BUFFER_SIZE = 320
VALIDATION_N_GROUP_PER_SAMPLING = 2
VALIDATION_N_BATCH_PER_SAMPLING = 80

# Buffer size calculation
# Max GPU memory usage = buffer_size * batch_size * num_columns * 4 bits
#                      = (480 + 320) * 3072 * 1000 * 4
#                      = 9830400000 bits
#                      = 9.16 GB

In [ ]:
lf = pl.scan_parquet(TRAINSET_DATA_PATH)
n_samples = lf.select(pl.len()).collect().item()

parquet = pq.ParquetFile(TRAINSET_DATA_PATH, memory_map=True, buffer_size=10)
all_groups = list(range(0, parquet.num_row_groups))
train_groups = random.sample(all_groups, int(TRAINING_SAMPLE_FRAC * len(all_groups)))
val_groups = list(set(all_groups) - set(train_groups))

dataset_train = Dataset(
    parquet=parquet,
    input_cols=INPUT_COLUMNS,
    target_cols=OUTPUT_COLUMNS,
    n_samples=int(n_samples / len(all_groups) * len(train_groups)),
    groups=train_groups,
    buffer_size=TRAINING_BUFFER_SIZE,
    batch_size=BATCH_SIZE,
    n_group_per_sampling=TRAINING_N_GROUP_PER_SAMPLING,
    n_batch_per_sampling=TRAINING_N_BATCH_PER_SAMPLING,
)
dataset_val = Dataset(
    parquet=parquet,
    input_cols=INPUT_COLUMNS,
    target_cols=OUTPUT_COLUMNS,
    n_samples=int(n_samples / len(all_groups) * len(val_groups)),
    groups=val_groups,
    buffer_size=VALIDATION_BUFFER_SIZE,
    batch_size=BATCH_SIZE,
    n_group_per_sampling=VALIDATION_N_GROUP_PER_SAMPLING,
    n_batch_per_sampling=VALIDATION_N_BATCH_PER_SAMPLING,
)

In [ ]:
model = src.pytorch.models.MLP()

trainloader = dataset_train.to_dataloader()
valloader = dataset_val.to_dataloader()

model, best_weights, loss = src.pytorch.models.utils.train(
    model=model,
    dataloaders={"Training": trainloader, "Validation": valloader},
    n_epochs=N_EPOCHS,
)
src.visualization.performance.loss_curve(loss, close=False)

In [ ]:
# Save weights if needed
MODEL_WEIGHTS_PATH = "./model.pt"

src.pytorch.models.utils.save_weights(
    model=model,
    model_path=MODEL_WEIGHTS_PATH,
)

In [ ]:
# Load data to generate submission file

TESTSET_DATA_PATH = "/home/data/test.arrow"
TESTSET_PREDICTION_WEIGHTS_PATH = "/home/data/sample_submission.arrow"

# df_submission = pl.read_ipc(TESTSET_DATA_PATH).to_pandas()
# df_weights = pl.read_ipc(TESTSET_PREDICTION_WEIGHTS_PATH).to_pandas()

# Load model to generate submission file

# model = src.pytorch.models.MLP()
# src.pytorch.models.utils.load_model(model, MODEL_WEIGHTS_PATH)
# model.to(src.env.DEVICE)

In [ ]:
# If you want to submit the predictions

# src.pytorch.data.kaggle.output_compressed_parquet(
#     model=model,
#     df=df_submission,
#     weights=df_weights,
# )